In [1]:
# @Author Hsieh Lee Hsun
# @Date 2018/11/16
# @Description: Weather data from EPA data 
import pickle as pk
from pprint import pprint
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from datetime import timedelta
from random import randint
import json
import math
import geopy

In [ ]:
# Read the data from pk which gets from mongodb
data = list()
for i in range(4, 6, 1):
    with open("All_PM25_data_201" + str(i) + ".pk","rb") as file:
        data.append(pk.load(file))

In [ ]:
# All data from 2014 ~ 2016
#Construct a dictionary contains datas of every hours
overall_data = list()
#for feature in feature_set:
#    overall_data[feature] = list()
for i in range(2):
    for index in tqdm(range(len(data[i]))):
        d = data[i][index]
        is_pm25 = False
        if d['feature'] == "PM2.5":
            is_pm25 = True
        for hour in d['hr_data']:
            individual_data = dict()
            if hour['hr_flag']:
                individual_data['data'] = 0
            else:
                individual_data['data'] = hour['hr_v']
            individual_data['date'] = datetime(d['date']['year'], d['date']['month'], 
                                               d['date']['day'], hour['hr_t'])
            individual_data['station'] = d['station']
            individual_data['feature'] = d['feature']
            overall_data.append(individual_data)

In [ ]:
# Dump all the data to the pickle
# Release the memory space
del data
with open('All_PM25_data_DataFrame_list_2014_to_2015.pk', 'wb') as file:
    pk.dump(overall_data, file)
del overall_data[:]
del overall_data

In [ ]:
# Read the data from pk which gets from mongodb
with open("All_PM25_data_2017_8_9.pk","rb") as file:
    data = pk.load(file)

In [ ]:
# All data from 2014 ~ 2016
#Construct a dictionary contains datas of every hours
overall_data = list()
#for feature in feature_set:
#    overall_data[feature] = list()
for index in tqdm(range(len(data))):
    d = data[index]
    is_pm25 = False
    if d['feature'] == "PM2.5":
        is_pm25 = True
    for hour in d['hr_data']:
        individual_data = dict()
        if hour['hr_flag']:
            individual_data['data'] = 0
        else:
            individual_data['data'] = hour['hr_v']
        individual_data['date'] = datetime(d['date']['year'], d['date']['month'],
                                           d['date']['day'], hour['hr_t'])
        individual_data['station'] = d['station']
        individual_data['feature'] = d['feature']
        overall_data.append(individual_data)

In [ ]:
# Dump all the data to the pickle
# Release the memory space
del data
with open('All_PM25_data_DataFrame_list_2017_8_9.pk', 'wb') as file:
    pk.dump(overall_data, file)
del overall_data[:]
del overall_data

In [ ]:
with open('All_PM25_data_DataFrame_list_2014_to_2015.pk', 'rb') as file:
    overall_data = pk.load(file)
with open('All_PM25_data_DataFrame_list_2016_to_2017.pk', 'rb') as file:
    overall_data_2 = pk.load(file)
with open('All_PM25_data_DataFrame_list_2017_8_9.pk', 'rb') as file:
    overall_data_3 = pk.load(file)
overall_data += overall_data_2
overall_data += overall_data_3
del overall_data_2[:]
del overall_data_2
del overall_data_3[:]
del overall_data_3

In [ ]:
# Reshape all the data and use 'date' as row index and 'station' as column
date = pd.date_range(start='1/1/2014', end='10/1/2017', freq='H', closed='left')
overall_dataframe = pd.DataFrame(overall_data).pivot_table(index='date', 
                                                           columns=['station', 'feature'], 
                                                           values='data', 
                                                           aggfunc='first').reindex(date)
# Reference: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.pivot.html
# Serialize the result
with open("All_PM25_data_DataFrame_training.pk","wb") as file:
    pk.dump(overall_dataframe, file)
# End of All_PM25_data_DataFrame.pk

In [25]:
#Preprocess the 2014 ~ 2017 training data including normalize
#Read the data from pickle
with open("All_PM25_data_DataFrame_training.pk","rb") as file:
    test = pk.load(file)

interpolate_idx = pd.Index(['AMB_TEMP', 'PM10', 'PM2.5', 'RH', 'WIND_SPEED', 'WS_HR'])

def f(x):
    return (x.apply(lambda x : x if x > 0 else 0))

test = test.drop('馬祖東引', axis=1)
test = test.drop([("陽明", "WIND_DIREC"), ("陽明", "WIND_SPEED"), ("淡水", "WIND_DIREC"), ("淡水", "WIND_SPEED")], axis=1)
test = test.apply(f)
test = test.replace('null', np.nan)
test.loc[:, (slice(None), interpolate_idx)] = \
    test.loc[:, (slice(None), interpolate_idx)].replace(0, np.nan)


In [26]:
# Choose the valid row of data
valid_col = pd.Index(['AMB_TEMP', 'PM10', 'PM2.5', 'RH', 'WIND_DIREC', 'WIND_SPEED', 'WD_HR', 'WS_HR'])
valid_test_set = test.loc[:, (slice(None), valid_col)]

# Findout sites without wind direction but PM2.5
idx1 = valid_test_set.loc[:, (slice(None), 'WIND_DIREC')].columns.get_level_values(0)
idx2 = valid_test_set.loc[:, (slice(None), 'PM2.5')].columns.get_level_values(0)
without_direc = idx2.difference(idx1)
print(without_direc)

# Print two examples of data
print(valid_test_set.loc[:, '淡水'].head())
print(valid_test_set.loc[:, '花蓮'].head())

Index(['三重', '大同', '淡水', '陽明'], dtype='object', name='station')
feature              AMB_TEMP  PM10  PM2.5  RH
2014-01-01 00:00:00       NaN  46.0   26.0 NaN
2014-01-01 01:00:00       NaN  45.0   28.0 NaN
2014-01-01 02:00:00       NaN  50.0   31.0 NaN
2014-01-01 03:00:00       NaN  60.0   35.0 NaN
2014-01-01 04:00:00       NaN  57.0   36.0 NaN
feature              AMB_TEMP  PM10  PM2.5    RH  WD_HR  WIND_DIREC  \
2014-01-01 00:00:00      17.0  53.0   36.0  87.0  251.0       244.0   
2014-01-01 01:00:00      17.0  51.0   34.0  87.0  266.0       267.0   
2014-01-01 02:00:00      17.0  42.0   27.0  86.0  249.0       240.0   
2014-01-01 03:00:00      17.0  37.0   23.0  85.0  255.0       272.0   
2014-01-01 04:00:00      17.0  32.0   21.0  84.0  239.0       237.0   

feature              WIND_SPEED  WS_HR  
2014-01-01 00:00:00         1.9    1.6  
2014-01-01 01:00:00         1.5    1.3  
2014-01-01 02:00:00         1.6    1.2  
2014-01-01 03:00:00         1.2    1.2  
2014-01-01 04:00:00   

In [10]:
valid_test_set.loc[:, (slice(None), 'WIND_DIREC')].columns.get_level_values(0)

Index(['三義', '中壢', '中山', '二林', '仁武', '冬山', '前金', '前鎮', '南投', '古亭', '善化', '嘉義',
       '土城', '埔里', '基隆', '士林', '大園', '大寮', '大里', '安南', '宜蘭', '小港', '屏東', '崙背',
       '左營', '平鎮', '彰化', '復興', '忠明', '恆春', '斗六', '新店', '新港', '新營', '新竹', '新莊',
       '朴子', '松山', '板橋', '林口', '林園', '桃園', '楠梓', '橋頭', '永和', '汐止', '沙鹿', '淡水',
       '湖口', '潮州', '竹山', '竹東', '線西', '美濃', '臺南', '臺東', '臺西', '花蓮', '苗栗', '菜寮',
       '萬華', '萬里', '西屯', '觀音', '豐原', '金門', '關山', '陽明', '頭份', '馬公', '馬祖', '鳳山',
       '麥寮', '龍潭'],
      dtype='object', name='station')

In [28]:
'''
Construct the cos and sine component of Direction and the PM2.5_diff and Normalize as well

The base of the data

PM2.5 500
PM10 500
WS_HR 30
WD_HR(cos, sin) 1
WIND_SPEED 30
WIND_DIREC(cos, sin) 1
AMB_TEMP 50
RH 100

'''

valid_test_set.loc[:, (slice(None), 'PM2.5')] = valid_test_set.loc[:, (slice(None), 'PM2.5')] / 500
valid_test_set.loc[:, (slice(None), 'PM10')] = valid_test_set.loc[:, (slice(None), 'PM10')] / 500
valid_test_set.loc[:, (slice(None), 'WS_HR')] = valid_test_set.loc[:, (slice(None), 'WS_HR')] / 30
valid_test_set.loc[:, (slice(None), 'WIND_SPEED')] = valid_test_set.loc[:, (slice(None), 'WIND_SPEED')] / 30
valid_test_set.loc[:, (slice(None), 'AMB_TEMP')] = valid_test_set.loc[:, (slice(None), 'AMB_TEMP')] / 50
valid_test_set.loc[:, (slice(None), 'RH')] = valid_test_set.loc[:, (slice(None), 'RH')] / 100

/home/sean/.local/lib/python3.5/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [29]:
without_wind = False

for station in set(valid_test_set.columns.get_level_values(0)):
    
    without_wind = (without_direc == station).any()
    
    # Fill the missing data
    if without_wind:
        valid_test_set.loc[:, (station, 'WIND_DIREC_COS')] = 0
        valid_test_set.loc[:, (station, 'WIND_DIREC_SIN')] = 0
        valid_test_set.loc[:, (station, 'WD_HR_COS')] = 0
        valid_test_set.loc[:, (station, 'WD_HR_SIN')] = 0
        valid_test_set.loc[:, (station, 'WIND_SPEED')] = 0
        valid_test_set.loc[:, (station, 'WS_HR')] = 0
        if station == '淡水' or station == '大同':
            valid_test_set.loc[:, (station, 'AMB_TEMP')] = 0
            valid_test_set.loc[:, (station, 'RH')] = 0
        
    else:
        try:
            valid_test_set.loc[:, (station, 'WIND_DIREC_COS')] = valid_test_set.loc[:, (station, 'WIND_DIREC')].apply(lambda x : x / 180 * np.pi).apply(np.cos)
            valid_test_set.loc[:, (station, 'WIND_DIREC_SIN')] = valid_test_set.loc[:, (station, 'WIND_DIREC')].apply(lambda x : x / 180 * np.pi).apply(np.sin)        
            valid_test_set.loc[:, (station, 'WD_HR_COS')] = valid_test_set.loc[:, (station, 'WD_HR')].apply(lambda x : x / 180 * np.pi).apply(np.cos)
            valid_test_set.loc[:, (station, 'WD_HR_SIN')] = valid_test_set.loc[:, (station, 'WD_HR')].apply(lambda x : x / 180 * np.pi).apply(np.sin)
        except:
            print(station, " Failed!")
        
    valid_test_set.loc[:, (station, 'PM2.5_diff')] = valid_test_set.loc[:, (station, 'PM2.5')].diff()

# Drop the Wind direction
valid_test_set = valid_test_set.drop('WIND_DIREC', axis = 1, level = 1)
valid_test_set = valid_test_set.drop('WD_HR', axis = 1, level = 1)
valid_test_set = valid_test_set.interpolate(limit_direction = "both")
valid_test_set = valid_test_set.sort_index(1)
# valid_test_set.head()
print(valid_test_set.isna().any().any())

/home/sean/.local/lib/python3.5/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/sean/.local/lib/python3.5/site-packages/pandas/core/indexing.py:630: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


False


In [31]:
valid_test_set.loc[:, '臺南'].tail()

feature,AMB_TEMP,PM10,PM2.5,PM2.5_diff,RH,WD_HR_COS,WD_HR_SIN,WIND_DIREC_COS,WIND_DIREC_SIN,WIND_SPEED,WS_HR
2017-09-30 19:00:00,0.6,0.074,0.032,-0.008,0.75,0.438371,-0.898794,0.669131,-0.743145,0.036667,0.036667
2017-09-30 20:00:00,0.6,0.072,0.034,0.002,0.77,0.882948,-0.469472,0.999194,0.040132,0.033333,0.023333
2017-09-30 21:00:00,0.6,0.076,0.036,0.002,0.78,0.224951,-0.974370,0.173648,-0.984808,0.053333,0.036667
2017-09-30 22:00:00,0.6,0.066,0.038,0.002,0.79,0.406737,-0.913545,-0.017452,-0.999848,0.033333,0.026667
2017-09-30 23:00:00,0.6,0.066,0.044,0.006,0.80,0.848048,0.529919,0.891007,0.453990,0.026667,0.013333


In [32]:
with open('./knn_dtw.json', 'r') as f:
    knn_dtw_data = json.load(f)
with open('./knn_ed.json', 'r') as f:
    knn_ed_data = json.load(f)
with open('./station.pk', 'rb') as f:
    station = pk.load(f)
# Form two dictionaries to project from id to name and name from id

station_to_num = dict()
num_to_station = dict()
for data in station:
    station_to_num[data['Sitename']] = str(data['id'])
    num_to_station[str(data['id'])] = data['Sitename']

In [37]:
# Straight store individual station data

station_list = list()

training_date = pd.date_range(start='1/1/2014', end='10/1/2016', freq='H', closed='left')
val_date = pd.date_range(start='10/1/2016', end='10/1/2017', freq='H', closed='left')

for station in set(valid_test_set.columns.get_level_values(0)):
    with open("./Seq2Seq_history_df/All_PM25_DataFrame_training_" + station_to_num[station] + ".pk", "wb") as file:
        pk.dump(valid_test_set.loc[training_date, station], file)
    with open("./Seq2Seq_history_df/All_PM25_DataFrame_label_" + station_to_num[station] + ".pk", "wb") as file:
        pk.dump(valid_test_set.loc[training_date, (station, 'PM2.5')], file)
    with open("./Seq2Seq_history_df/All_PM25_DataFrame_val_" + station_to_num[station] + ".pk", "wb") as file:
        pk.dump(valid_test_set.loc[val_date, station], file)
    with open("./Seq2Seq_history_df/All_PM25_DataFrame_val_label_" + station_to_num[station] + ".pk", "wb") as file:
        pk.dump(valid_test_set.loc[val_date, (station, 'PM2.5')], file)
    
    station_list.append(station_to_num[station])
        
with open("./Seq2Seq_history_df/station_list.pk", "wb") as file:
    pk.dump(station_list, file)

In [39]:
# Prepare the data into (Batch_size, 77, 6) according to the result of dtw and ed

station_num = 0
avaliable_station = list()

overall_ed_dtw_station = list()

training_date = pd.date_range(start='1/1/2014', end='10/1/2016', freq='H', closed='left')
val_date = pd.date_range(start='10/1/2016', end='10/1/2017', freq='H', closed='left')

for station in set(valid_test_set.columns.get_level_values(0)):
    
    # Read the ed and dtw into a list forming a index
    ed_station_list = list()
    ed_dtw_station = list()
    # The current station itself is in the list of ed
    ed_station_list.append(station)
    # Overall ed dtw station
    ed_dtw_station.append(station_to_num[station])
    
    dtw_station_list = list()
    
    ed = knn_ed_data[station_to_num[station]]
    dtw = knn_dtw_data[station_to_num[station]]

    for i in range(3):
        ed_station_list.append(num_to_station[ed[i]])
        dtw_station_list.append(num_to_station[dtw[i]])
        ed_dtw_station.append(ed[i])
        ed_dtw_station.append(dtw[i])
    
    ed_idx = pd.Index(ed_station_list, name='station')
    dtw_idx = pd.Index(dtw_station_list, name='station')
    
#     if station_to_num[station] == "4":
#         print(dtw_idx)
#         print(valid_test_set.loc[:, (dtw_idx, feature_index)])
    
    # Get those station
    test_set_npy = np.concatenate((valid_test_set.loc[training_date, ed_idx].values, 
                                   valid_test_set.loc[training_date, dtw_idx].values), axis = 1)
    
    pm25 = valid_test_set.loc[training_date, (station, 'PM2.5')].values
    
    val_set_npy = np.concatenate((valid_test_set.loc[val_date, ed_idx].values, 
                                  valid_test_set.loc[val_date, dtw_idx].values), axis = 1)
    
    val_pm25 = valid_test_set.loc[val_date, (station, 'PM2.5')].values
    
    # If the shape doesn't match
    if(test_set_npy.shape[1] != 77):
        print(test_set_npy.shape)
        print(ed_idx)
        print(dtw_idx)
        print(station, " failed !")
        continue
        
    #Prepare two empty numpy arrays
    training_set = np.empty([test_set_npy.shape[0] - 12, 77, 6], dtype=np.float32)
    label_set = np.empty([test_set_npy.shape[0] - 12, 6], dtype=np.float32)
    
    val_set = np.empty([val_set_npy.shape[0] - 12, 77, 6], dtype=np.float32)
    val_label_set = np.empty([val_set_npy.shape[0] - 12, 6], dtype=np.float32)
    
    for i in range(test_set_npy.shape[0] - 12):
        training_set[i] = test_set_npy[i : i + 6].T
        label_set[i] = pm25[i + 6 : i + 12]
    
    for i in range(val_set_npy.shape[0] - 12):
        val_set[i] = val_set_npy[i : i + 6].T
        val_label_set[i] = val_pm25[i + 6 : i + 12]
    
    # Save those data
    np.save('history_npy/{}.npy'.format(station_to_num[station]), training_set)
    np.save('history_npy/{}_label.npy'.format(station_to_num[station]), label_set)
    
    # Save those data
    np.save('history_npy/{}_val.npy'.format(station_to_num[station]), val_set)
    np.save('history_npy/{}_val_label.npy'.format(station_to_num[station]), val_label_set)
    
    # Calculate the number of stations
    station_num += 1
    avaliable_station.append(station_to_num[station])
    
    # Overall station
    overall_ed_dtw_station.append(ed_dtw_station)
    
    print(station, " Ok !")

# Overall number of station
print("station num", station_num)

with open('./history_npy/station_list.pk', 'wb') as file:
    pk.dump(avaliable_station, file)
    
with open("overall_ed_dtw_station.pk", "wb") as file:
    pk.dump(overall_ed_dtw_station, file)
    

橋頭  Ok !
中山  Ok !
恆春  Ok !
平鎮  Ok !
楠梓  Ok !
淡水  Ok !
臺南  Ok !
菜寮  Ok !
馬公  Ok !
士林  Ok !
麥寮  Ok !
新店  Ok !
基隆  Ok !
嘉義  Ok !
頭份  Ok !
沙鹿  Ok !
西屯  Ok !
苗栗  Ok !
林園  Ok !
陽明  Ok !
馬祖  Ok !
萬里  Ok !
安南  Ok !
潮州  Ok !
龍潭  Ok !
萬華  Ok !
冬山  Ok !
大寮  Ok !
線西  Ok !
觀音  Ok !
大同  Ok !
新莊  Ok !
古亭  Ok !
桃園  Ok !
美濃  Ok !
臺東  Ok !
埔里  Ok !
新港  Ok !
朴子  Ok !
林口  Ok !
花蓮  Ok !
關山  Ok !
二林  Ok !
鳳山  Ok !
金門  Ok !
中壢  Ok !
崙背  Ok !
小港  Ok !
三義  Ok !
屏東  Ok !
永和  Ok !
南投  Ok !
豐原  Ok !
新竹  Ok !
前金  Ok !
復興  Ok !
竹山  Ok !
汐止  Ok !
斗六  Ok !
新營  Ok !
湖口  Ok !
仁武  Ok !
大園  Ok !
宜蘭  Ok !
左營  Ok !
三重  Ok !
松山  Ok !
土城  Ok !
前鎮  Ok !
彰化  Ok !
大里  Ok !
臺西  Ok !
板橋  Ok !
忠明  Ok !
善化  Ok !
竹東  Ok !
station num 76


In [42]:
print(valid_test_set.loc[:, "大同"].head())
print(valid_test_set.loc[:, "大同"].head().values)

feature              AMB_TEMP   PM10  PM2.5  PM2.5_diff  RH  WD_HR_COS  \
2014-01-01 00:00:00         0  0.198  0.080      -0.014   0          0   
2014-01-01 01:00:00         0  0.188  0.066      -0.014   0          0   
2014-01-01 02:00:00         0  0.180  0.068       0.002   0          0   
2014-01-01 03:00:00         0  0.172  0.044      -0.024   0          0   
2014-01-01 04:00:00         0  0.156  0.052       0.008   0          0   

feature              WD_HR_SIN  WIND_DIREC_COS  WIND_DIREC_SIN  WIND_SPEED  \
2014-01-01 00:00:00          0               0               0           0   
2014-01-01 01:00:00          0               0               0           0   
2014-01-01 02:00:00          0               0               0           0   
2014-01-01 03:00:00          0               0               0           0   
2014-01-01 04:00:00          0               0               0           0   

feature              WS_HR  
2014-01-01 00:00:00      0  
2014-01-01 01:00:00      0  

In [43]:

# Return the lat, lon and height    
    
def get_height(st_no):
    with open('height121.json') as df:
        data = json.loads(df.read())
    raw_temp = []
    for i in range(len(data)):
        raw_elem = []
        if(data[i]['id'] == st_no):
            raw_elem.append(data[i]['y'])
            raw_elem.append(data[i]['x'])
            raw_elem.append(data[i]['elevation'])
            raw_elem.append(data[i]['latitude'])
            raw_elem.append(data[i]['longitude'])
            raw_temp.append(raw_elem)
    

    
    h_matr_l = int(math.sqrt(len(raw_temp)))
    h_lat_lon = np.empty((h_matr_l, h_matr_l, 2))
    h_matr = np.empty((h_matr_l, h_matr_l))
    
    
    for elem in raw_temp:
        h_matr[elem[0] - 1][elem[1] - 1] = elem[2]
        h_lat_lon[elem[0] - 1][elem[1] - 1][0] = elem[3]
        h_lat_lon[elem[0] - 1][elem[1] - 1][1] = elem[4]
        
    return h_matr / 1000, h_lat_lon


In [44]:
# Calculate all the distance of the map of cnn around all the sites

from geopy import distance

with open('station.pk', 'rb') as file:
    station = pk.load(file)
    
station_dict = dict()

for s in station:
    station_dict[s['id'] - 1] = (s['lat'], s['lng'])

lat_lon_dict = dict()
height_dict = dict()
idw_denominator = dict()

for site in range(76):
    #print(site)
    h_matr, h_lat_lon = get_height(site + 1)
    d = np.zeros((11, 11, 76))
    for i in range(11):
        for j in range(11):
            for s in station_dict:
                d[i, j, s] = distance.distance(station_dict[s], h_lat_lon[i][j]).km
                
    lat_lon_dict[str(site)] = d
    height_dict[str(site)] = h_matr
    
# Becuase site 47 is exactly on the grid so it will lead to the error of divided by zero    
lat_lon_dict['46'][5, 5, 46] = 0.001
    
with open('distance_of_all_sites.pk', 'wb') as file:
    pk.dump(lat_lon_dict, file)

with open('Height_of_all_sites.pk', 'wb') as file:
    pk.dump(height_dict, file)

In [ ]:
def calculate_idw_value(p_matrix, x_list, y_list, xypair):
    p = 2.0
    # weight_matrix = np.zeros([len(xypair), p_matrix.shape[0], p_matrix.shape[1]])
    weight_matrix = np.zeros([len(xypair), p_matrix.shape[1], p_matrix.shape[2]])
    sum_weight_matrix = np.zeros([p_matrix.shape[1], p_matrix.shape[2]])
    for m in range(p_matrix.shape[1]):
        for n in range(p_matrix.shape[2]):
            for i in range(len(xypair)):
                if (m, n) in xypair:
                    weight_matrix[i, m, n] = 1
                    sum_weight_matrix[m, n] = 1
                else:
                    weight_matrix[i, m, n] = 1 / (math.sqrt((m - x_list[i])**2 + (n - y_list[i])**2) ** p)
                    sum_weight_matrix[m, n] += 1 / (math.sqrt((m - x_list[i])**2 + (n - y_list[i])**2) ** p)
 
    for z in range(p_matrix.shape[0]):
        sum_matrix = np.zeros([p_matrix.shape[1], p_matrix.shape[2]])
        for i in range(len(xypair)):
            sum_matrix += weight_matrix[i] * p_matrix[z, x_list[i], y_list[i]]
        for j in range(len(xypair)):
            sum_matrix[x_list[j], y_list[j]] = p_matrix[z, x_list[j], y_list[j]]
        p_matrix[z] = sum_matrix / sum_weight_matrix
        
    return p_matrix

According to the paper, the elevation and the value of pm2.5 has the following relationship.

$\frac{1}{e^{\frac{H - H_{st}}{H_{st}}}}$ 

In [45]:
# Denominator of the IDW
# It should contain all the distance of all the sites relative to the target grid and sum them all.
denominator = np.empty((76, 11, 11))
for site in range(76):
    for i in range(11):
        for j in range(11):
            denominator[site, i, j] = ((1 / np.square(lat_lon_dict[str(site)][i , j])).sum())
                            #* np.exp((height_dict[str(site)][i, j] - height_dict[str(site)][5, 5]) / height_dict[str(site)][5, 5]))
            

# Numerator of IDW
# It should contain all the distance of all the sites relative to the target grid.
# The difference between denominator and numerator is that numerator stores the individual data of the distances
# to the target grid and denominator sums them all.
numerator = np.empty((76, 11, 11, 76))
for site in range(76):
    for i in range(11):
        for j in range(11):
            numerator[site, i, j] = 1 / np.square(lat_lon_dict[str(site)][i , j])

In [63]:
from tqdm import trange

site_map = np.zeros((valid_test_set.loc[training_date].shape[0], 76, 11, 11))

print(site_map.shape)

# r stands for row
for idx, pm25 in valid_test_set.loc[training_date, (slice(None), 'PM2.5')].iterrows():
    all_pm25_list = pm25.values
    for site in range(76):
        for i in range(11):
            for j in range(11):
                a = (all_pm25_list * numerator[site, i, j]).sum()
                site_map[r, site, i, j] = a / denominator[site, i, j]

(24096, 76, 11, 11)


In [64]:
with open('training_cnn_site_map.pk', 'wb') as file:
    pk.dump(site_map, file)

In [65]:
from tqdm import trange

site_map = np.zeros((valid_test_set.loc[val_date].shape[0], 76, 11, 11))

print(site_map.shape)

# r stands for row
for idx, pm25 in valid_test_set.loc[val_date, (slice(None), 'PM2.5')].iterrows():
    all_pm25_list = pm25.values
    for site in range(76):
        for i in range(11):
            for j in range(11):
                a = (all_pm25_list * numerator[site, i, j]).sum()
                site_map[r, site, i, j] = a / denominator[site, i, j]

(8760, 76, 11, 11)


In [67]:
with open('val_cnn_site_map.pk', 'wb') as file:
    pk.dump(site_map, file)

KeyboardInterrupt: 

In [68]:
with open('val_cnn_site_map.pk', 'rb') as file:
    val_cnn_site_map = pk.load(file)

In [70]:
cnn_val = val_cnn_site_map[:, 0]
print(cnn_val.shape)
cnn_val = np.expand_dims(cnn_val, axis = 3)
print(cnn_val.shape)
cnn_val = np.delete(cnn_val, slice(0, 7), axis = 0)
print(cnn_val.shape)
cnn_val = np.delete(cnn_val, slice(-6, -1), axis = 0)
print(cnn_val.shape)
cnn_val = np.delete(cnn_val, -1, axis = 0)
print(cnn_val.shape)

(8760, 11, 11)
(8760, 11, 11, 1)
(8753, 11, 11, 1)
(8748, 11, 11, 1)
(8747, 11, 11, 1)


In [75]:
cnn_val = val_cnn_site_map[:, 0:1]

In [76]:
cnn_val.shape

(8760, 1, 11, 11)

In [77]:
cnn_val[7:-6].shape

(8747, 1, 11, 11)

In [79]:
a = np.zeros((44,1))